In [1]:
import pandas as pd
import requests as rq
import numpy as np
import os, sys
import pandas as pd
import requests as rq
import numpy as np
import os, sys
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, r2_score
from random import seed
from random import randrange
from csv import reader
from math import sqrt

In [2]:
# just run one time
#os.mkdir('data')

In [3]:
#Download data
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'
path = os.path.join('data','wine_red.csv')
response = rq.get(url)
with open(path, 'wb') as f:
    f.write(response.content)

In [4]:
# read and load data
dataset = pd.read_csv(path, delimiter=';')
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
#Separate label to data point
X = dataset.loc[:, dataset.columns != 'quality']
y = dataset.loc[:, dataset.columns == 'quality']

#normalize by Min-Max scaling 
mm_scaler = preprocessing.MinMaxScaler()
X = mm_scaler.fit_transform(X)

X = pd.DataFrame(X)
#randomrize the data
X, y = shuffle(X, y, random_state=0)
#get the test set 10000 examples = 55.9 % of total dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.408329, random_state=0)
test = np.hstack((X_test,y_test))

In [6]:
print('the size of evaluate set is of :',y_test.shape[0])
print('the size of train set is of :',y_train.shape[0])

the size of evaluate set is of : 2000
the size of train set is of : 2898


In [7]:
#case 1 training dataset 100 examples
X_train100, Y_train100 = shuffle(X_train,y_train, n_samples=100, random_state=0)
print('First case. the size of train set is of :',X_train100.shape[0])

#case 2 training dataset 1000 examples
X_train1000, Y_train1000 = shuffle(X_train,y_train, n_samples=1000, random_state=2)
print('Third case. the size of train set is of :',X_train1000.shape[0])

#case 2 training dataset 2898 examples
X_train2898, Y_train2898 = shuffle(X_train,y_train, n_samples=2883, random_state=3)
print('Fourth case. the size of train set is of :',X_train2898.shape[0])

First case. the size of train set is of : 100
Third case. the size of train set is of : 1000
Fourth case. the size of train set is of : 2883


In [44]:
# Make a prediction with coefficients
def predict(rows, coefficients):
    y_predicted = coefficients[0]
    for index in range(len(rows)-1):
        y_predicted += coefficients[index + 1] * rows[index]
    return y_predicted

In [41]:
# Estimate linear regression coefficients using stochastic gradient descent
def sgd(train, l_rate, n_epoch):
    loss = []
    coef = [0.0 for i in range(len(train[0]))]# initialize weights in zero
    for epoch in range(n_epoch):
        for rows in train:
            y_predicted = predict(rows, coef) # get predicted value
            error = y_predicted - rows[-1] #calculate error
            coef[0] = coef[0] - l_rate * error#b0(t+1) = b0(t) - learning_rate * error(t)
            for i in range(len(rows)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * rows[i]# b1(t+1) = b1(t) - learning_rate * error(t) * x1(t)
            # print(l_rate, n_epoch, error)
        loss.append(error)
    return coef, loss

In [42]:
# Linear Regression Algorithm With Stochastic Gradient Descent
def linear_regression_sgd(train, test, l_rate, n_epoch):
    predictions = list()
    coef, loss = sgd(train, l_rate, n_epoch)# learning the weights in training data
    for row in test:
        yhat = predict(row, coef)# make p
        predictions.append(yhat)
    return(predictions), loss

In [45]:
# evaluate algorithm to fist case
l_rate = 0.001
n_epoch = 500
train = np.hstack((X_train100,Y_train100))
y_pred, loss = linear_regression_sgd(train, test, l_rate, n_epoch)
print("Mean squared error: %.2f"% mean_squared_error(y_test, y_pred))

Mean squared error: 0.65


In [36]:
# evaluate algorithm to fist case
l_rate = 0.01
n_epoch = 500
train = np.hstack((X_train1000,Y_train1000))
y_pred, loss = linear_regression_sgd(train, test, l_rate, n_epoch)
print("Mean squared error: %.2f"% mean_squared_error(y_test, y_pred))

Mean squared error: 0.59


In [38]:
# evaluate algorithm to fist case
l_rate = 0.01
n_epoch = 500
train = np.hstack((X_train2898,Y_train2898))
y_pred, loss = linear_regression_sgd(train, test, l_rate, n_epoch)
print("Mean squared error: %.2f"% mean_squared_error(y_test, y_pred))

Mean squared error: 0.58


## Conclusions
The above result shows how to aument the trainin data we will get less MSE. It is due to get more data, the model can adjust better to points. Happens the same to seconds task using sklearn LinearRegressor.